In [1]:
import sys
import baggianalysis as ba
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance_matrix
from icp import icp

In [2]:
arms = 30
N_per_arm = 30
# we want to retain the central bead
ids = [0, ]
# and the tips of the arms
ids += [a * N_per_arm + N_per_arm for a in range(arms)]
myfilter = ba.FilterByFunction(lambda p: p.index in ids)

parser = ba.GenericOxDNAParser("ba_topology.dat")
trajectory = ba.FullTrajectory(parser)
trajectory.add_filter(myfilter)
trajectory.initialise_from_trajectory_file("trajectory.dat")

In [3]:
# compute the average distance between the tips and the centre
trajectory.reset()
system = trajectory.next_frame()
dist_avg_per_tip = np.zeros(arms)
n_confs = 0
while system != None:
    n_confs += 1
    centre = system.particle_by_id(0).position
    for i, p_idx in enumerate(ids[1:]):
        distance = system.particle_by_id(p_idx).position - centre
        dist_avg_per_tip[i] += np.sqrt(np.dot(distance, distance))

    system = trajectory.next_frame()

dist_avg_per_tip /= n_confs
dist_avg = np.mean(dist_avg_per_tip)
print("Average distances:", dist_avg_per_tip)
print("Average distance:", dist_avg)

Average distances: [12.23464495 12.32709907 12.30352984 12.05665088 12.1298742  12.31562993
 12.26249684 12.1334117  12.28552248 11.98653209 12.26499654 12.28407311
 12.28362966 12.10006092 12.34939585 12.17344873 12.29649194 12.24668078
 12.28852312 11.91040085 12.17655181 12.09880717 12.11550542 12.29112547
 12.17697587 12.08147486 11.93853086 12.38213588 12.25555659 12.52767926]
Average distance: 12.209247888907521


In [4]:
trajectory.reset()
system = trajectory.next_frame()
n_confs = 0
base_positions = None
while system != None:
    n_confs += 1
    
    positions = []
    for p in system.particles():
        if p.index == 0:
            centre = p.position
        else:
            positions.append(p.position)
    positions -= centre
    
    if base_positions is None:
        base_positions = np.copy(positions)
        dest = np.copy(base_positions)
    else:
        T, distances, iterations = icp(positions, base_positions)

        # make C a homogeneous representation of src
        C = np.ones((arms, 4))
        C[:,0:3] = np.copy(positions)
        # transform C according to the change of coordinates obtained with the icp procedure
        C = np.dot(T, C.T).T
        # obtain the new coordinates
        positions_corrected = C[:,:3]
        
        # compute the distance matrix
        graph_weights = distance_matrix(base_positions, positions_corrected)
        # perform the linear assignment
        row_ind, col_ind = linear_sum_assignment(graph_weights)
        # reorder the array according to the result of the linear assignment
        positions_corrected_reordered = positions_corrected[col_ind,:]
        
        dest += positions_corrected_reordered
    
    system = trajectory.next_frame()

dest /= n_confs

In [5]:
dest_distance_matrix = distance_matrix(dest, dest)
cut_off = dest_distance_matrix.max()
dest_distance_matrix_norm = dest_distance_matrix / cut_off

first_piece = dest_distance_matrix_norm < 0.5
second_piece = (dest_distance_matrix_norm >= 0.5) & (dest_distance_matrix_norm < 1.0)
third_piece = dest_distance_matrix_norm >= 1.0

weight_matrix = np.piecewise(
    dest_distance_matrix_norm, 
    [first_piece, second_piece, third_piece],
    [lambda r: 1 - 6*r**2 + 6*r**3, lambda r: 2 - 6*r + 6*r**2 - 2*r**3, 0.]
)
# see https://stackoverflow.com/a/54637261/5140209
weight_matrix = np.broadcast_to(weight_matrix[:,:,np.newaxis,np.newaxis], (arms, arms, 3, 3))

# now we compute D_inv for each reference point
dest_diff_matrix = dest[:, np.newaxis, :] - dest
# first we perform an outer product on the last axis
D = dest_diff_matrix[:,:,:,np.newaxis] * dest_diff_matrix[:,:,np.newaxis,:] * weight_matrix
# then we sum over all the points (i.e. on the second axis)
D = np.sum(D, axis=1)
# and invert the matrices
D_inv = np.linalg.inv(D)

In [6]:
def compute_F(system):
    src = []
    for p in system.particles():
        if p.index == 0:
            centre = p.position
        else:
            src.append(p.position)
    src -= centre
    
    T, distances, iterations = icp(src, dest)

    # make C a homogeneous representation of src
    C = np.ones((arms, 4))
    C[:,0:3] = np.copy(src)
    # transform C according to the change of coordinates obtained with the icp procedure
    C = np.dot(T, C.T).T
    # obtain the new coordinates
    src_corrected = C[:,:3]
    
    # compute the distance matrix
    graph_weights = distance_matrix(dest, src_corrected)
    # perform the linear assignment
    row_ind, col_ind = linear_sum_assignment(graph_weights)
    # reorder the array according to the result of the linear assignment
    src_corrected_reordered = src_corrected[col_ind,:]
    
    # compute A for each point
    src_diff_matrix = src_corrected_reordered[:, np.newaxis, :] - src_corrected_reordered
    A = src_diff_matrix[:,:,:,np.newaxis] * dest_diff_matrix[:,:,np.newaxis,:] * weight_matrix
    # then we sum over all the points (i.e. along the second axis)
    A = np.sum(A, axis=1)

    F = A @ D_inv

    return F

In [7]:
trajectory.reset()
system = trajectory.next_frame()
confs = 0
J_tot = []
I_tot = []
J_avg = []
I_avg = []
J_from_F_avg = []
I_from_F_avg = []
while system != None:
    confs += 1
    if confs % 100 == 0:
        print("Analysed %s confs" % confs)

    # and now we can compute the Cauchy-Green strain tensor
    F = compute_F(system)
    F_T = np.transpose(F, axes=(0,2,1))
    C = np.matmul(F_T, F)

    # from the invariants of C we obtain J and I
    J = np.sqrt(np.linalg.det(C))
    I = np.trace(C, axis1=1, axis2=2) / J**(2./3.)
    
    J_tot += list(J)
    I_tot += list(I)
        
    J_avg.append(np.average(J))
    I_avg.append(np.average(I * J**(2./3.)) / J_avg[-1]**(2./3.))
    #I_avg.append(np.average(I))
    
    F_avg = np.average(F, axis=0)
    C_avg = F_avg.T @ F_avg
    J_from_F_avg.append(np.sqrt(np.linalg.det(C_avg)))
    I_from_F_avg.append(np.trace(C_avg) / J_from_F_avg[-1]**(2./3.))

    system = trajectory.next_frame()

np.savetxt("J.dat", J_tot)
np.savetxt("I.dat", I_tot)

np.savetxt("J_avg.dat", J_avg)
np.savetxt("I_avg.dat", I_avg)

np.savetxt("J_from_F_avg.dat", J_from_F_avg)
np.savetxt("I_from_F_avg.dat", I_from_F_avg)

Analysed 100 confs
Analysed 200 confs
Analysed 300 confs
Analysed 400 confs
Analysed 500 confs


In [8]:
def make_pmf(x, remove_last=False):
    hist, bins = np.histogram(x, bins='auto', density=True)
    bins = bins[:-1] + (bins[1] - bins[0]) / 2.
    result = np.column_stack((bins, hist))
    # get rid of the last points
    if remove_last != False:
        if len(bins) < 2 * remove_last:
            remove_last = len(bins) / 5
        result = result[result[:,1] > 0][:-remove_last]
    else:
        result = result[result[:,1] > 0]
    result[:,1] = -np.log(result[:,1])

    return result

def fit_J(x, m, D0, b):
    return m * (x - D0)**2. + b

def fit_I(x, m, b):
    return m * x + b

pmf_J = make_pmf(J_tot)
pmf_I = make_pmf(I_tot, 50)

np.savetxt("J_pmf.dat", pmf_J)
np.savetxt("I_pmf.dat", pmf_I)

pmf_J_avg = make_pmf(J_avg)
pmf_I_avg = make_pmf(I_avg)

np.savetxt("J_avg_pmf.dat", pmf_J_avg)
np.savetxt("I_avg_pmf.dat", pmf_I_avg)

pmf_J_from_F_avg = make_pmf(J_from_F_avg)
pmf_I_from_F_avg = make_pmf(I_from_F_avg)

np.savetxt("J_from_F_avg_pmf.dat", pmf_J_from_F_avg)
np.savetxt("I_from_F_avg_pmf.dat", pmf_I_from_F_avg)